In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    # !wandb login
    !pip install umap-learn
    !pip install umap-learn[plot]
    !pip install holoviews
    !pip install -U ipykernel
    !pip install joypy
    # !pip install umap-learn
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
#     #sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/data')
#     #sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/models')
#     sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
#     sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
#     sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
#     sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Clustering_Results/Results')
#     sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Clustering_Results/Figures')
#     sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Clustering_Result')
elif "zmqshell" in str(get_ipython()):
    print("Running on MIRP")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics/Jaguas"
    root_path = "media/mirp_ai/Seagate Desktop Drive/Jaguas_2018"
else:
    import pathlib
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    print("Running local")

Running on MIRP


In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from six.moves import xrange
import datetime
import gc
import pandas as pd
import joypy

from scipy import signal
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler as Normalizer

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torchaudio.transforms as audio_transform


#from ResidualStack import ResidualStack
#from Residual import Residual

from Jaguas_DataLoader_rainless import SoundscapeData
from Models import ConvAE as AE
from AE_training_functions import TestModel, TrainModel
from AE_Clustering import AE_Clustering 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()

from datetime import timedelta
import wandb
from wandb import AlertLevel

!pip install umap-learn
!pip install umap-learn[plot]
import umap
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

import seaborn as sns
import pickle as pkl

import random
def _set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).
 
    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
_set_seed(1024)

In [ ]:
model = "AE"
identifier = "batch_size_14_num_hiddens_64_"
day = 27
hour = 19
month = 6
date_format = f"day_{day}_hour_{hour}"
folder = f"{model}_No_rain"
labels_rainless = torch.load(f"temporal/Features/{folder}/{model}_labels_{date_format}_No_rain_Audios_Jaguas.pth",  map_location=torch.device('cpu'))
data_rainless = torch.load(f"temporal/Features/{folder}/{model}_features_{date_format}_No_rain_Audios_Jaguas.pth",  map_location=torch.device('cpu'))
# labels_rain = torch.load("temporal/features/")
# data_rain = torch.load("temporal/features/")

X_n = data_rainless.numpy()
#Standard Scaler
scaler = StandardScaler().fit(X_n)
X_scaled = scaler.transform(X_n)
#Normalizing
X_norm = Normalizer().fit_transform(X_n)
#PCA
n_components = 30
X_PCA = PCA(n_components=n_components).fit_transform(X_norm)
# X_PCA = StandardScaler().fit_transform(X_PCA)
# X_PCA = Normalizer().fit_transform(X_PCA)
#TSNE
X_TSNE = TSNE(n_components=2, learning_rate="auto", init='random', random_state=0).fit_transform(X_PCA)
X_TSNE_Norm = Normalizer().fit_transform(X_TSNE)

#UMAP
reducer = umap.UMAP(min_dist=0.9)
X_UMAP = reducer.fit_transform(X_PCA)
X_UMAP_Norm = Normalizer().fit_transform(X_UMAP)

In [ ]:
metadata = "hour"
y_n = labels_rainless[metadata].numpy()

X_ = [X_scaled, X_norm, X_PCA, X_TSNE, X_UMAP]
Feats = ["Scaled", "Norm", f"PCA{n_components}", "TSNE", "UMAP"]
for enum, x in enumerate(X_):
    sns.jointplot(x=X_[enum][:,0], y= X_[enum][:,1], hue=y_n, joint_kws={'alpha': 0.03}) 
#     plt.savefig(f"Projection_{model}_{Feats[enum]}_{metadata}_alpha_03")

In [ ]:
# X_scaled_df = pd.DataFrame(X_scaled)
X_PCA_df = pd.DataFrame(X_PCA)
X_norm_df = pd.DataFrame(X_norm)
X_UMAP_df = pd.DataFrame(X_UMAP)
# X_scaled_df.to_csv("X_Scaled.csv")
X_UMAP_df.to_csv("X_UMAP.csv")
X_norm_df.to_csv("X_norm.csv")
X_PCA_df.to_csv("X_PCA.csv")

for dist in [0.2, 0.4, 0.6, 0.8, 1]  :
    for n in [15, 30, 45, 100, 150, 250, 500, 100]:
        reducer = umap.UMAP(n_neighbors=n, min_dist=dist)
        X_UMAP = reducer.fit_transform(X_PCA)
        X_ = Normalizer().fit_transform(X_UMAP)
        sns.jointplot(x=X_[:,0], y= X_[:,1], hue=y_n, joint_kws={'alpha': 0.03})
        plt.savefig(f"Projection_AE_UMAP_{metadata}_alpha_03_neighboors_{reducer.n_neighbors}_dist_{reducer.min_dist}.png")        

## PCA

In [ ]:
n_components = 120
X_PCA_ = PCA(n_components=n_components).fit(X_norm)
X_PCA = X_PCA_.transform(X_norm)
plt.figure(figsize=(10,8))
plt.plot(range(0, n_components), X_PCA_.explained_variance_ratio_.cumsum())
plt.title("Explained variance by components")
plt.xlabel("Number of components")
plt.ylabel("Cummulative Explained Variance ")
plt.axhline(y=0.9, c="r", linestyle="--")

## TSNE

In [ ]:
X_TSNE = TSNE(n_components=2, learning_rate="auto", init='random', random_state=0).fit_transform(X_scaled)

In [ ]:
a= np.arange(X_TSNE.shape[0])
a = np.random.choice(a, size=84820)
new_embedding = X_TSNE[a]
y_n2 = y_n[a]

plt.scatter(
    new_embedding[:, 0],
    new_embedding[:, 1], c=y_n2)
plt.gca().set_aspect('equal', 'datalim')
plt.title('TSNE projection of Jaguas dataset', fontsize=24);

In [ ]:
new_embedding.shape

## UMAP

In [ ]:
reducer = umap.UMAP(n_components=2)

In [ ]:
X_UMAP = reducer.fit_transform(X_PCA)

In [ ]:
X_desUMAP = reducer.inverse_transform(X_UMAP)

In [ ]:
X_desUMAP.shape

In [ ]:
a= np.arange(X_UMAP.shape[0])
a = np.random.choice(a, size=84820)
new_embedding = X_UMAP[a]
y_n2 = y_n[a]

plt.scatter(
    new_embedding[:, 0],
    new_embedding[:, 1], c=y_n2)
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of Jaguas dataset', fontsize=24);


# Feature Exploration

In [ ]:
X = X_norm.copy()

In [ ]:
X = 